In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
from pymc3 import __version__
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(__version__))

Running on PyMC3 v3.8


In [3]:
from pymc3 import get_data

In [4]:
# Import radon data
srrs2 = pd.read_csv(get_data('srrs2.dat'))
srrs2.columns = srrs2.columns.map(str.strip)
srrs_mn = srrs2[srrs2.state=='MN'].copy()

In [14]:
srrs2 = pd.read_csv(get_data('srrs2.dat'))
print(srrs2)

       idnum state state2  stfips    zip  region  typebldg  floor  room  \
0          1    AZ     AZ       4  85920       1         1      1     2   
1          2    AZ     AZ       4  85920       1         0      9     0   
2          3    AZ     AZ       4  85924       1         1      1     3   
3          4    AZ     AZ       4  85925       1         1      1     3   
4          5    AZ     AZ       4  85932       1         1      1     1   
...      ...   ...    ...     ...    ...     ...       ...    ...   ...   
12772  12773    R5     WI      55  54660       4         1      0     4   
12773  12774    R5     WI      55  54660       4         1      1     3   
12774  12775    R5     WI      55  54660       4         1      1     3   
12775  12776    R5     WI      55  54660       4         1      1     3   
12776  12777    R5     WI      55  54741       4         1      1     1   

      basement  ... stoptm startdt  stopdt activity  pcterr       adjwt  \
0            N  ...   11

In [5]:
srrs_mn['fips'] = srrs_mn.stfips*1000 + srrs_mn.cntyfips
cty = pd.read_csv(get_data('cty.dat'))
cty_mn = cty[cty.st=='MN'].copy()
cty_mn[ 'fips'] = 1000*cty_mn.stfips + cty_mn.ctfips

In [7]:
srrs_mn = srrs_mn.merge(cty_mn[['fips', 'Uppm']], on='fips')
srrs_mn = srrs_mn.drop_duplicates(subset='idnum')
u = np.log(srrs_mn.Uppm)

In [8]:
n = len(srrs_mn)

In [9]:
srrs_mn.head()

,idnum,state,state2,stfips,zip,region,typebldg,floor,room,basement,...,stopdt,activity,pcterr,adjwt,dupflag,zipflag,cntyfips,county,fips,Uppm
0,5081,MN,MN,27,55735,5,1,1,3,N,...,12288,2.2,9.7,1146.499190,1,0,1,AITKIN,27001,0.502054
1,5082,MN,MN,27,55748,5,1,0,4,Y,...,12088,2.2,14.5,471.366223,0,0,1,AITKIN,27001,0.502054
2,5083,MN,MN,27,55748,5,1,0,4,Y,...,21188,2.9,9.6,433.316718,0,0,1,AITKIN,27001,0.502054
3,5084,MN,MN,27,56469,5,1,0,4,Y,...,123187,1.0,24.3,461.623670,0,0,1,AITKIN,27001,0.502054
4,5085,MN,MN,27,55011,3,1,0,4,Y,...,13088,3.1,13.8,433.316718,0,0,3,ANOKA,27003,0.428565


In [10]:
srrs_mn.county = srrs_mn.county.map(str.strip)
mn_counties = srrs_mn.county.unique()
counties = len(mn_counties)
county_lookup = dict(zip(mn_counties, range(len(mn_counties))))

In [11]:
county = srrs_mn['county_code'] = srrs_mn.county.replace(county_lookup).values
radon = srrs_mn.activity
srrs_mn['log_radon'] = log_radon = np.log(radon + 0.1).values
floor_measure = srrs_mn.floor.values

In [18]:
len(log_radon)

919